# 139_176 Importazioni ed esportazioni per paese e merce Ateco 2007

_vedi anche SDMX1.ipynb_


In [ ]:
https://sdmx.istat.it/SDMXWS/rest/data/139_176

# Backup

In [1]:
from sdmx import Client

# Inizializza il client ISTAT
istat = Client("Istat")

# STEP 1 – Carica il dataflow e ottieni la struttura associata
df_id = "139_176"
dataflows = istat.dataflow()
df = dataflows.dataflow[df_id]

# Nome leggibile robusto
if df.name.localizations:
    nome_df = (
        df.name.localizations.get("it") or
        df.name.localizations.get("en") or
        next(iter(df.name.localizations.values()))
    )
else:
    nome_df = str(df.name)

print(f"\n🧩 Dataflow ID: {df.id}")
print(f"📘 Nome: {nome_df}")
print(f"📎 DSD ID: {df.structure.id}")


🧩 Dataflow ID: 139_176
📘 Nome: Importazioni ed esportazioni per paese e merce Ateco 2007
📎 DSD ID: DCSP_COEIMPEX1


## analisi struttura dataset

In [3]:
# STEP 2 – Scarica la struttura dati
dsd_msg = istat.datastructure(df.structure.id)
dsd = dsd_msg.structure[0]


In [37]:
dsd

<DataStructureDefinition IT1:DCSP_COEIMPEX1(1.0): Import and export by country and commodity Nace 2007>

In [45]:
# STEP 3 – Elenca dimensioni e codelist associate
print("\n📐 Dimensioni e relative Codelist:")
for dim in dsd.dimensions:
    cl = dim.local_representation.enumerated
    cl_id = cl.id if cl else "—"
    print(f" - {dim.id} → {cl_id}")


📐 Dimensioni e relative Codelist:
 - FREQ → CL_FREQ
 - REF_AREA → CL_COEWEB_ITTER107
 - DATA_TYPE → CL_COEWEB_TIPO_DATO_COEIMPEX
 - CPA_ATECO2007_COE → CL_COEWEB_ATECO_2007_MERCE
 - PARTNER_COUNTRY → CL_COEWEB_GEONOM
 - TIME_PERIOD → —


| Dimensione          | Descrizione                           | Codelist                                     |
| ------------------- | ------------------------------------- | -------------------------------------------- |
| `FREQ`              | Frequenza (es. annuale, trimestrale)  | `CL_FREQ`                                    |
| `REF_AREA`          | Area di riferimento (es. IT, regioni) | `CL_COEWEB_ITTER107`                         |
| `DATA_TYPE`         | Tipo di dato (valori, indici, ecc.)   | `CL_COEWEB_TIPO_DATO_COEIMPEX`               |
| `CPA_ATECO2007_COE` | Codice merceologico Ateco 2007        | `CL_COEWEB_ATECO_2007_MERCE`                 |
| `PARTNER_COUNTRY`   | Paese partner                         | `CL_COEWEB_GEONOM`                           |
| `TIME_PERIOD`       | Periodo temporale                     | — *(gestito con `startPeriod`, `endPeriod`)* |


In [46]:
# STEP 4 – Elenca i valori ammessi per ciascuna codelist
print("\n📋 Contenuto delle Codelist:")
codelists = dsd_msg.codelist

for dim in dsd.dimensions:
    cl = dim.local_representation.enumerated
    cl_id = cl.id if cl else None
    if cl_id and cl_id in codelists:
        print(f"\n🔹 {dim.id} ({cl_id}):")
        for code in codelists[cl_id]:
            if code.name.localizations:
                label = (
                    code.name.localizations.get("it") or
                    code.name.localizations.get("en") or
                    next(iter(code.name.localizations.values()))
                )
            else:
                label = ""
            print(f"   - {code.id:<15} {label}")


📋 Contenuto delle Codelist:

🔹 FREQ (CL_FREQ):
   - A               annuale
   - B               business (non supportato)
   - D               giornaliero
   - E               event (non supportato)
   - H               semestrale
   - M               mensile
   - Q               trimestrale
   - W               settimanale

🔹 REF_AREA (CL_COEWEB_ITTER107):
   - 00              Raggruppamento a 107 province
   - IT              Italia
   - ITCDE           Centro-nord
   - ITCD            Nord
   - N_2             centro area metropolitana
   - N_MET_AR_INH_GE50001 periferia area metropolitana e comuni con 50.001 ab. e più
   - N_INH_OTH_UN5000 altri comuni fino a 50.000 ab. (diversi dai comuni periferia area metropolitana)
   - ITC             Nord-ovest
   - ITC1            Piemonte
   - ITC11           Torino
   - 001001          Agliè
   - 001002          Airasca
   - 001003          Ala di Stura
   - 001004          Albiano d'Ivrea
   - 001005          Alice Superiore
   - 001006

## estrazione dati

####  verificare i valori validi per ciascuna dimensione

In [47]:
def stampa_codelist(codelist_id, filtro=None):
    msg = istat.codelist(codelist_id)
    cl = list(msg.codelist.values())[0]
    for code in cl.items.values():
        if filtro is None or filtro(code):
            loc = code.name.localizations
            name = loc.get('it') or loc.get('en') or next(iter(loc.values()), code.id)
            print(f"{code.id} – {name}")


In [49]:
stampa_codelist('CL_FREQ')

A – annuale
B – business (non supportato)
D – giornaliero
E – event (non supportato)
H – semestrale
M – mensile
Q – trimestrale
W – settimanale


In [7]:

stampa_codelist('CL_COEWEB_TIPO_DATO_COEIMPEX')

IV – importazioni - valore (milioni di euro)
EV – esportazioni - valore (milioni di euro)
TBV – saldo commerciale - valore (milioni di euro)
ISAV – importazioni - valore destagionalizzato  - modello mondo (milioni di euro)
ESAV – esportazioni - valore destagionalizzato - modello mondo (milioni di euro)
TBSAV – saldo commerciale - valore destagionalizzato - modello mondo (milioni di euro)


In [8]:
stampa_codelist('CL_COEWEB_ATECO_2007_MERCE', filtro=lambda c: c.id.startswith('C'))


C – prodotti delle attivita' manifatturiere
CA – prodotti alimentari, bevande e tabacco
CB – prodotti tessili, abbigliamento, pelli e accessori
CC – legno e prodotti in legno; carta e stampa
CD – coke e prodotti petroliferi raffinati
CE – sostanze e prodotti chimici
CF – articoli farmaceutici, chimico-medicinali e botanici
CG – articoli in gomma e materie plastiche, altri prodotti della lavorazione di minerali non metalliferi
CH – metalli di base e prodotti in metallo, esclusi macchine e impianti
CI – computer, apparecchi elettronici e ottici
CJ – apparecchi elettrici
CK – macchinari ed apparecchi n.c.a.
CL – mezzi di trasporto
CM – prodotti delle altre attività manifatturiere


In [14]:
stampa_codelist("CL_COEWEB_ITTER107", filtro=lambda c: "italia" in c.name.localizations.get("it", "").lower())


IT – Italia
013040 – Campione d'Italia
T03028 – Distretto di Campione d'Italia
067026 – Isola del Gran Sasso d'Italia
063075 – San Vitaliano
082016 – Campofelice di Fitalia
083082 – San Salvatore di Fitalia
IT_C – Italia (comuni capoluogo)
IT_NC – Italia (comuni non capoluogo)


In [9]:
stampa_codelist('CL_COEWEB_GEONOM', filtro=lambda c: 'cina' in c.name.localizations.get('it', '').lower())


CN – Cina


In [50]:
import xml.etree.ElementTree as ET

# Percorso del nuovo file XML SDMX caricato
file_path = "D:/193_176.sdmx"

# Carica e analizza la struttura principale
tree = ET.parse(file_path)
root = tree.getroot()

# Namespace SDMX 2.1
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Conta elementi principali
series_count = len(root.findall(".//gen:Series", ns))
obs_count = len(root.findall(".//gen:Obs", ns))

# Estrai esempio di chiavi e valori da una Series
example_series = root.find(".//gen:Series", ns)
series_key = {
    s.attrib["id"]: s.attrib["value"]
    for s in example_series.findall("./gen:SeriesKey/gen:Value", ns)
}
first_obs = example_series.find(".//gen:Obs", ns)
obs_time = first_obs.find("./gen:ObsDimension", ns).attrib["value"]
obs_value = first_obs.find("./gen:ObsValue", ns).attrib.get("value")

# Riassunto
{
    "Totale serie": series_count,
    "Totale osservazioni": obs_count,
    "Chiavi esempio": series_key,
    "Prima osservazione": {
        "TIME_PERIOD": obs_time,
        "value": obs_value
    }
}


{'Totale serie': 47,
 'Totale osservazioni': 9919,
 'Chiavi esempio': {'FREQ': 'M',
  'REF_AREA': 'ITTOT',
  'DATA_TYPE': 'ESAV',
  'CPA_ATECO2007_COE': '0010',
  'PARTNER_COUNTRY': 'EU'},
 'Prima osservazione': {'TIME_PERIOD': '', 'value': '5908'}}

## Codice: genera chiave SDMX

In [52]:
chiave = costruisci_chiave_sdmx(
    freq="M",
    area="IT",               # Campania
    tipo_dato="ESAV",       # Verificato nella codelist
    ateco="C",           # Aggregazione valida
    partner="CN"               # Cina
)

response = istat.data(
    resource_id="139_176",
    key=chiave,
    params={"startPeriod": "2015", "endPeriod": "2024"}
)

df = response.to_pandas()
print(df.head())


HTTPError: 404 Client Error: Not Found for url: https://esploradati.istat.it/SDMXWS/rest/data/139_176/M.IT.ESAV.C.CN?startPeriod=2015&endPeriod=2024

In [54]:
import xml.etree.ElementTree as ET

# Percorso del file caricato da curl
file_path = "D:/139_176.xml"

# Parsing del file XML
tree = ET.parse(file_path)
root = tree.getroot()

# Namespace SDMX
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Trova le osservazioni e verifica la presenza di TIME_PERIOD
obs_with_time = []
for series in root.findall(".//gen:Series", ns):
    keys = {v.attrib["id"]: v.attrib["value"] for v in series.findall(".//gen:SeriesKey/gen:Value", ns)}
    for obs in series.findall(".//gen:Obs", ns):
        time = obs.find("gen:ObsDimension", ns).attrib.get("value", "")
        value = obs.find("gen:ObsValue", ns).attrib.get("value", "")
        obs_with_time.append({
            **keys,
            "TIME_PERIOD": time,
            "value": value
        })

# Converte in DataFrame
import pandas as pd
df = pd.DataFrame(obs_with_time)

df.head(20)


,FREQ,MERCE_ATECO_2007,PAESE_PARTNER,ITTER107,TIPO_DATO,TIME_PERIOD,value
0,M,0010,EU,ITTOT,ESAV,1993-01,5908
1,M,0010,EU,ITTOT,ESAV,1993-10,6544
2,M,0010,EU,ITTOT,ESAV,1993-11,6541
3,M,0010,EU,ITTOT,ESAV,1993-12,6782
4,M,0010,EU,ITTOT,ESAV,1993-02,6074
5,M,0010,EU,ITTOT,ESAV,1993-03,5889
6,M,0010,EU,ITTOT,ESAV,1993-04,6270
7,M,0010,EU,ITTOT,ESAV,1993-05,6020
8,M,0010,EU,ITTOT,ESAV,1993-06,6090
9,M,0010,EU,ITTOT,ESAV,1993-07,6180


In [20]:
import xml.etree.ElementTree as ET

# Carica il file XML SDMX
tree = ET.parse("D:/193_176.sdmx")
root = tree.getroot()

# Spazio dei nomi SDMX
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Estrai tutte le serie e le osservazioni
data = []
for series in root.findall(".//gen:Series", ns):
    serie_attribs = {
        s.attrib['id']: s.attrib['value']
        for s in series.findall("./gen:SeriesKey/gen:Value", ns)
    }

    for obs in series.findall(".//gen:Obs", ns):
        time = obs.find("./gen:ObsDimension", ns).attrib['value']
        value_elem = obs.find("./gen:ObsValue", ns)
        value = value_elem.attrib.get('value') if value_elem is not None else None

        row = {**serie_attribs, "TIME_PERIOD": time, "value": value}
        data.append(row)

import pandas as pd
df = pd.DataFrame(data)

df



,FREQ,REF_AREA,DATA_TYPE,CPA_ATECO2007_COE,PARTNER_COUNTRY,TIME_PERIOD,value
0,M,ITTOT,ESAV,0010,EU,,5908
1,M,ITTOT,ESAV,0010,EU,,6074
2,M,ITTOT,ESAV,0010,EU,,5889
3,M,ITTOT,ESAV,0010,EU,,6270
4,M,ITTOT,ESAV,0010,EU,,6020
...,...,...,...,...,...,...,...
9914,Q,ITTOT,EV,0010,WORLD,,155138
9915,Q,ITTOT,EV,0010,WORLD,,160740
9916,Q,ITTOT,EV,0010,WORLD,,147063
9917,Q,ITTOT,EV,0010,WORLD,,160567


In [22]:
tree.getroot()

<Element '{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}GenericData' at 0x0000011A31DE7240>

In [25]:
import xml.etree.ElementTree as ET
import pandas as pd

# Carica il file XML
file_path = "D:/193_176.sdmx"
tree = ET.parse(file_path)
root = tree.getroot()

# Namespace SDMX
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Parsing delle osservazioni
data = []
for series in root.findall(".//gen:Series", ns):
    serie_attribs = {
        s.attrib["id"]: s.attrib["value"]
        for s in series.findall("./gen:SeriesKey/gen:Value", ns)
    }
    for obs in series.findall(".//gen:Obs", ns):
        time_elem = obs.find("./gen:ObsDimension", ns)
        value_elem = obs.find("./gen:ObsValue", ns)
        time = time_elem.attrib["value"] if time_elem is not None else None
        value = value_elem.attrib["value"] if value_elem is not None else None

        row = {**serie_attribs, "TIME_PERIOD": time, "value": value}
        data.append(row)

df = pd.DataFrame(data)

# Filtri richiesti:
# - solo esportazioni (es. 'ESAV' o 'EV')
# - solo Cina e WORLD
# - solo CPA Ateco C10 e C11 (codici tipo 0010, 0011)

df = df[df["DATA_TYPE"].isin(["ESAV", "EV"])]
df = df[df["PARTNER_COUNTRY"].isin(["CN", "WORLD"])]
df = df[df["CPA_ATECO2007_COE"].isin(["0010", "0011"])]

# Ordina per tempo se disponibile
df = df.sort_values(by="TIME_PERIOD")

df



,FREQ,REF_AREA,DATA_TYPE,CPA_ATECO2007_COE,PARTNER_COUNTRY,TIME_PERIOD,value
777,M,ITTOT,ESAV,0010,WORLD,,10765
8676,Q,ITE4,EV,0010,WORLD,,5718
8677,Q,ITE4,EV,0010,WORLD,,6674
8678,Q,ITE4,EV,0010,WORLD,,5463
8679,Q,ITE4,EV,0010,WORLD,,5540
...,...,...,...,...,...,...,...
7453,Q,ITC4,EV,0010,WORLD,,28943
7454,Q,ITC4,EV,0010,WORLD,,26826
7455,Q,ITC4,EV,0010,WORLD,,28814
7457,Q,ITC4,EV,0010,WORLD,,28897


In [26]:
import requests

url = "https://esploradati.istat.it/SDMXWS/rest/data/139_176/ALL.ALL.ALL.ALL.ALL?startPeriod=2022&endPeriod=2022"

with open("D:/139_176_2022.xml", "wb") as f:
    f.write(requests.get(url).content)


In [29]:
import xml.etree.ElementTree as ET
import pandas as pd

# Percorso file XML
file_path = "D:/139_176_2022.xml"

# Namespace SDMX
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Parsing XML
tree = ET.parse(file_path)
root = tree.getroot()

data = []
for series in root.findall(".//gen:Series", ns):
    serie_attribs = {
        s.attrib["id"]: s.attrib["value"]
        for s in series.findall("./gen:SeriesKey/gen:Value", ns)
    }
    for obs in series.findall(".//gen:Obs", ns):
        time_elem = obs.find("./gen:ObsDimension", ns)
        value_elem = obs.find("./gen:ObsValue", ns)
        time = time_elem.attrib["value"] if time_elem is not None else None
        value = value_elem.attrib["value"] if value_elem is not None else None
        row = {**serie_attribs, "TIME_PERIOD": time, "value": value}
        data.append(row)

df = pd.DataFrame(data)

# Filtri richiesti
df = df[df["DATA_TYPE"].isin(["EV", "ESAV"])]
df = df[df["PARTNER_COUNTRY"].isin(["CN", "WORLD"])]
df = df[df["CPA_ATECO2007_COE"].isin(["0010", "0011"])]
df = df.sort_values(by="TIME_PERIOD")

# Salva in CSV
df.to_csv("D:/esportazioni_C10_C11_CN_WORLD.csv", index=False, encoding="utf-8")
print("✅ File salvato in: D:/esportazioni_C10_C11_CN_WORLD.csv")


ParseError: syntax error: line 1, column 0 (<string>)

In [36]:
import xml.etree.ElementTree as ET
import pandas as pd

# Rileggo il file ora confermato valido
file_path = "D:/193_176.sdmx"
tree = ET.parse(file_path)
root = tree.getroot()

# Namespace SDMX 2.1
ns = {
    "mes": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "gen": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic"
}

# Estrai tutte le serie e le osservazioni
data = []
for series in root.findall(".//gen:Series", ns):
    serie_attribs = {
        s.attrib["id"]: s.attrib["value"]
        for s in series.findall("./gen:SeriesKey/gen:Value", ns)
    }
    for obs in series.findall(".//gen:Obs", ns):
        time_elem = obs.find("./gen:ObsDimension", ns)
        value_elem = obs.find("./gen:ObsValue", ns)
        time = time_elem.attrib["value"] if time_elem is not None else None
        value = value_elem.attrib["value"] if value_elem is not None else None
        row = {**serie_attribs, "TIME_PERIOD": time, "value": value}
        data.append(row)

df = pd.DataFrame(data)

# Filtri:
df = df[df["DATA_TYPE"].isin(["EV", "ESAV"])]
df = df[df["PARTNER_COUNTRY"].isin(["CN", "WORLD"])]
df = df[df["CPA_ATECO2007_COE"].isin(["0010", "0011"])]
df = df.sort_values(by="TIME_PERIOD")
df

,FREQ,REF_AREA,DATA_TYPE,CPA_ATECO2007_COE,PARTNER_COUNTRY,TIME_PERIOD,value
777,M,ITTOT,ESAV,0010,WORLD,,10765
8676,Q,ITE4,EV,0010,WORLD,,5718
8677,Q,ITE4,EV,0010,WORLD,,6674
8678,Q,ITE4,EV,0010,WORLD,,5463
8679,Q,ITE4,EV,0010,WORLD,,5540
...,...,...,...,...,...,...,...
7453,Q,ITC4,EV,0010,WORLD,,28943
7454,Q,ITC4,EV,0010,WORLD,,26826
7455,Q,ITC4,EV,0010,WORLD,,28814
7457,Q,ITC4,EV,0010,WORLD,,28897
